# Tutorial: Block Partitioned Matrices

This tutorial introduces the `block_partitioned_matrices`, a library of
operations on hierarchically nested structured matrices.  The library builds
upon a base `Matrix` class. Here are some examples of its derived class:

*   `Tensor`: A wrapper around `torch.Tensor`.
*   `Identity`: Represents an identity matrix (implicitly).
*   `Zero`: Represents a zero matrix (implicitly).
*   `Vertical`: Represents a column vector of blocks.
*   `Diagonal`: Represents a block-diagonal matrix.
* ...

What sets this library apart from other matrix libraries is that matrices can be
nested within each other. For example, tensors can be stacked diagonally into a
`Diagonal` matrix of `Tensor` matrices. These can in turn be stacked vertically
into a `Vertical` matrix.

Here is a full list of the matrix classes:

In [1]:
import block_partitioned_matrices as bpm
# Print the last paragraph of the module docstring. It's automatically
# generated.
print(bpm.__doc__.split("\n\n")[-1])

Matrix
├── Tensor (also torch.Tensor)
├── Identity
│   └── ScaledIdentity
├── Zero
└── Ragged
    ├── Generic
    │   ├── Generic3x3
    │   ├── Vertical
    │   └── Horizontal
    ├── Symmetric2x2
    └── Tridiagonal
        ├── SymmetricTriDiagonal
        ├── LowerBiDiagonal
        │   ├── IdentityWithLowerDiagonal
        │   └── LowerDiagonal
        ├── UpperBiDiagonal
        │   ├── IdentityWithUpperDiagonal
        │   └── UpperDiagonal
        └── Diagonal



## 1. Introduction and Setup

In [2]:
import torch
from block_partitioned_matrices import (
    Tensor,
    Identity,
    Zero,
    Vertical,
    Generic,
    Diagonal,
    Symmetric2x2,
    Tridiagonal,
    LowerBiDiagonal,
)


# Helper function to create column vectors easily
def col(*args):
    return torch.tensor(args)[:, None]

## 2. Basic Components

Placeholder matrices like Zero, Identity, etc lets us represent certain matrices
without storing their content:

In [3]:
# Wrap a standard PyTorch tensor
t = Tensor(torch.tensor([[1.0, 2.0], [3.0, 4.0]]))
print("Wrapped Tensor:")
print(t.to_tensor())

# Identity and Zero matrices don't store full data
I = Identity(2)  # 2x2 Identity
Z = Zero((2, 2))  # 2x2 Zero

print("\nIdentity + Zero (converted to dense tensor):")
print((I + Z).to_tensor())

Wrapped Tensor:
tensor([[1., 2.],
        [3., 4.]])

Identity + Zero (converted to dense tensor):
tensor([[1., 0.],
        [0., 1.]])


Here is a typical way to construct a matrix (in this case, two column vectors stacked on top of each other):

In [4]:
# Construct a vertical vector from two smaller vectors
V = Vertical([col(1.0, 2.0), col(3.0, 4.0, 5.0)])

print("Vertical Vector properties:")
print(f"Shape: {V.shape}")
print(f"Number of blocks: {V.num_blocks()}")
print("Dense representation:")
print(V.to_tensor())

Vertical Vector properties:
Shape: (2, 1)
Number of blocks: 2
Dense representation:
tensor([[1.],
        [2.],
        [3.],
        [4.],
        [5.]])


Similarly, we can stack matrices into a 2x2 block diagonal matrix:

In [5]:
# Create a block diagonal matrix from two blocks
D = Diagonal([torch.eye(2), torch.ones(3, 3)])

print("\nBlock Diagonal Matrix:")
print(D.to_tensor())


Block Diagonal Matrix:
tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 1., 1.],
        [0., 0., 1., 1., 1.],
        [0., 0., 1., 1., 1.]])


The library supports standard linear algebra operations. These take advantage of
the structure of the matrices and return structured matrices as results:

*   **Arithmetic**: `+`, `-`
*   **Multiplication**: `@` (matmul)
*   **Inversion**: `.invert()`
*   **Solving**: `.solve(rhs)`

In [6]:
D2 = Diagonal([2 * torch.eye(2), 2 * torch.eye(3)])
D_sum = D + D2
print("Sum of two diagonal matrices:")
print(D_sum.to_tensor())

Sum of two diagonal matrices:
tensor([[3., 0., 0., 0., 0.],
        [0., 3., 0., 0., 0.],
        [0., 0., 3., 1., 1.],
        [0., 0., 1., 3., 1.],
        [0., 0., 1., 1., 3.]])


Multiply Diagonal matrix D by Vertical vector V.

In [7]:
result = D @ V
print("Matrix-Vector Multiplication (D @ V):")
print(result.to_tensor())

Matrix-Vector Multiplication (D @ V):
tensor([[ 1.],
        [ 2.],
        [12.],
        [12.],
        [12.]])


Invert a diagonal matrix (inverts each block individually)

In [8]:
D_inv = D2.invert()
print("Inverse of D2:")
print(D_inv.to_tensor())

# Verify inversion
should_be_identity = D2 @ D_inv
print("\nD2 @ D2_inv (Top-left block):")
print(should_be_identity.flat[0].to_tensor())

Inverse of D2:
tensor([[0.5000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.5000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.5000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.5000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.5000]])

D2 @ D2_inv (Top-left block):
tensor([[1., 0.],
        [0., 1.]])


Solve $D_2 x = V$. Since $D_2$ is diagonal, this solves block-wise.

In [9]:
x = D2.solve(V)

print("Solution x to D2 @ x = V:")
print(x.to_tensor())

# Verify solution
residual = (D2 @ x - V).to_tensor()
print(f"Residual norm: {residual.norm().item()}")

Solution x to D2 @ x = V:
tensor([[0.5000],
        [1.0000],
        [1.5000],
        [2.0000],
        [2.5000]])
Residual norm: 0.0


### 3. Nesting matrices

A `Generic` matrix is a grid of blocks, similar to how one might partition a
rectangular matrix on paper. Here is a 2x2 block matrix, each of whose blocks are torch Tensors:

In [10]:
G = Generic(
    [
        [torch.ones(2, 2), torch.zeros(2, 3)],
        [torch.zeros(3, 2), torch.eye(3)],
    ]
)
print("Generic 2x2 Block Matrix:")
print(G.to_tensor())

Generic 2x2 Block Matrix:
tensor([[1., 1., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]])


The power of the library comes from nesting matrices. For example, the blocks of
a `Generic` matrix can be `Diagonal` matrices, or any other `Matrix` subclass:

In [11]:
Nested = Generic(
    [
        [Diagonal([torch.eye(2), torch.eye(2)]), Zero((4, 2))],
        [Zero((2, 4)), Tensor(torch.ones(2, 2))],
    ]
)
print("Nested Block Matrix:")
print(Nested.to_tensor())

Nested Block Matrix:
tensor([[1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 1., 1.]])


## 3. Specialized Structures

Beyond just stacking blocks vertically, horizontally, and diagonally, the
library offers ways to compactly store symmetric and banded-diagonal matrices.

### 3.1 Symmetric 2x2 Block Matrix

Represents a matrix of the form:
$$
\begin{pmatrix}
A_{11} & A_{12} \\
A_{12}^T & A_{22}
\end{pmatrix}
$$

In [12]:
S = Symmetric2x2(torch.eye(2), 0.1 * torch.ones(2, 2), torch.eye(2) * 4)
print("Symmetric 2x2 Matrix:")
print(S.to_tensor())

# Confirm the matrix can be inverted.
torch.linalg.inv(S.to_tensor())

# Inversion uses Schur complement internally
S_inv = S.invert()
print("Inverse of Symmetric 2x2:")
print(S_inv.to_tensor())

Symmetric 2x2 Matrix:
tensor([[1.0000, 0.0000, 0.1000, 0.1000],
        [0.0000, 1.0000, 0.1000, 0.1000],
        [0.1000, 0.1000, 4.0000, 0.0000],
        [0.1000, 0.1000, 0.0000, 4.0000]])
Inverse of Symmetric 2x2:
tensor([[ 1.0051,  0.0051, -0.0253, -0.0253],
        [ 0.0051,  1.0051, -0.0253, -0.0253],
        [-0.0253, -0.0253,  0.2513,  0.0013],
        [-0.0253, -0.0253,  0.0013,  0.2513]])


### 3.2 Tridiagonal and Bi-diagonal Matrices

Structures like `LowerBiDiagonal`, `UpperBiDiagonal`, and `Tridiagonal` allow for solving systems using forward/backward substitution or LDU decompositions, which is $O(N)$ in the number of blocks rather than $O(N^3)$.

In [13]:
# Construct a Lower Bi-Diagonal Matrix
# [ D1  0 ]
# [ L1  D2]
L_mat = LowerBiDiagonal(
    diagonal_blocks=[torch.eye(2), torch.eye(2)], lower_blocks=[torch.ones(2, 2)]
)
print("Lower Bi-Diagonal Matrix:")
print(L_mat.to_tensor())

# Solve using forward substitution (implicitly)
rhs = Vertical([col(1.0, 1.0), col(2.0, 2.0)])
sol_L = L_mat.solve(rhs)
print("\nSolution to L_mat @ x = rhs:")
print(sol_L.to_tensor())

Lower Bi-Diagonal Matrix:
tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 1., 1., 0.],
        [1., 1., 0., 1.]])

Solution to L_mat @ x = rhs:
tensor([[1.],
        [1.],
        [0.],
        [0.]])


## 4. Matrix Decomposition

The library implements some basic factorization algorithms. These algorithms can handle nested matrices.

For example, we can decompose a `Tridiagonal` matrix of blocks into LDU (Lower-Diagonal-Upper). This lets use solve linear systems:

In [14]:
# Create a Tridiagonal Matrix
tri = Tridiagonal(
    [torch.eye(2) * 2, torch.eye(2) * 2, torch.eye(2) * 2],
    lower_blocks=[0.5 * torch.ones(2, 2), 0.5 * torch.ones(2, 2)],
    upper_blocks=[0.5 * torch.ones(2, 2), 0.5 * torch.ones(2, 2)],
)
print("Tridiagonal Matrix:")
print(tri.to_tensor())

# Perform LDU Decomposition
L, D, U = tri.LDU_decomposition()

print("\nDecomposed components:")
print(f"L type: {type(L)}")
print(f"D type: {type(D)}")
print(f"U type: {type(U)}")

# Verify correctness by applying to a vector
v = Vertical([col(1.0, 1.0), col(2.0, 2.0), col(3.0, 3.0)])

# Apply original matrix
tri_v = tri @ v

# Apply decomposed factors: L @ (D @ (U @ v))
LDU_v = L @ (D @ (U @ v))

print("\nDifference between tri @ v and L @ D @ U @ v:")
print((LDU_v - tri_v).to_tensor().norm().item())

Tridiagonal Matrix:
tensor([[2.0000, 0.0000, 0.5000, 0.5000, 0.0000, 0.0000],
        [0.0000, 2.0000, 0.5000, 0.5000, 0.0000, 0.0000],
        [0.5000, 0.5000, 2.0000, 0.0000, 0.5000, 0.5000],
        [0.5000, 0.5000, 0.0000, 2.0000, 0.5000, 0.5000],
        [0.0000, 0.0000, 0.5000, 0.5000, 2.0000, 0.0000],
        [0.0000, 0.0000, 0.5000, 0.5000, 0.0000, 2.0000]])

Decomposed components:
L type: <class 'block_partitioned_matrices.IdentityWithLowerDiagonal'>
D type: <class 'block_partitioned_matrices.Diagonal'>
U type: <class 'block_partitioned_matrices.IdentityWithUpperDiagonal'>

Difference between tri @ v and L @ D @ U @ v:
0.0
